# Tutorial

Kai Foerster, Amin Oueslati, Steve Kerr

## Introduction
Policy motivation: many institutions want to use something like ChatGPT but with their own domain knowledge <br>
Explain what a RAG chatbot is   <br>

### Next steps

	- Add HF token in .env

# Setup

* Install dependencies
* Configure an API key for Hugging Face

In [2]:
# install dependencies
!pipenv install langchain
!pipenv install sentence_transformers
!pipenv install chromadb
!pipenv install unstructured
!pipenv install chainlit
!pipenv install python-dotenv
!pipenv install bs4
!pipenv install tqdm
!pipenv install pandas
!pipenv install openpyxl
!pipenv install scikit-learn
!pipenv numpy

Loading .env environment variables...
Courtesy Notice: Pipenv found itself running within a virtual environment, so it will automatically use that environment, instead of creating its own for any project. You can set PIPENV_IGNORE_VIRTUALENVS=1 to force pipenv to ignore that environment and create its own instead. You can set PIPENV_VERBOSITY=-1 to suppress this warning.
Installing langchain...
Resolving langchain...
✔ Installation Succeeded
⠋ Installing langchain...
Installing dependencies from Pipfile.lock (b78d33)...
To activate this project's virtualenv, run pipenv shell.
Alternatively, run a command inside the virtualenv with pipenv run.
Loading .env environment variables...
Courtesy Notice: Pipenv found itself running within a virtual environment, so it will automatically use that environment, instead of creating its own for any project. You can set PIPENV_IGNORE_VIRTUALENVS=1 to force pipenv to ignore that environment and create its own instead. You can set PIPENV_VERBOSITY=-1 t

## Building a chatbot (no RAG)

In [3]:
import os
import chainlit as cl
from langchain import HuggingFaceHub, PromptTemplate, LLMChain
from dotenv import load_dotenv

2023-12-13 23:16:00 - Loaded .env file


In [4]:
load_dotenv()
HF_API_TOKEN = os.getenv('HF_API_TOKEN')

In [5]:
model_id = "tiiuae/falcon-7b-instruct"
conv_model = HuggingFaceHub(
    huggingfacehub_api_token=os.environ['HF_API_TOKEN'], 
    repo_id=model_id, 
    model_kwargs={"temperature":0.8,"max_length": 1000}
    )

/Users/steve/.local/share/virtualenvs/dl-tutorial-Si6gq9gB/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/steve/.local/share/virtualenvs/dl-tutorial-Si6gq9gB/lib/python3.11/site-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


In [6]:
template="""You are a helpful assistant that answers questions of the user.
{human_message}
"""

prompt=PromptTemplate(template=template, input_variables=["human_message"])

In [7]:
conv_chain = LLMChain(llm=conv_model, prompt=prompt, verbose=True)

In [8]:
print(conv_chain.run("How much does a cappucino cost at Pret a Manger in Berlin Mitte?"))



> Entering new LLMChain chain...
Prompt after formatting:
You are a helpful assistant that answers questions of the user.
How much does a cappucino cost at Pret a Manger in Berlin Mitte?


> Finished chain.
Today's currency exchange rate is USD to EUR = 1 EUR = $1.14. Therefore, the cost of a cappucino at Pret a Manger in Berlin Mitte is approximately $1.14.


### Hallucinations

In [9]:
print(conv_chain.run("How much does a cappucino cost at Pret a Manger in Berlin Mitte?"))



> Entering new LLMChain chain...
Prompt after formatting:
You are a helpful assistant that answers questions of the user.
How much does a cappucino cost at Pret a Manger in Berlin Mitte?


> Finished chain.
Today's currency exchange rate is USD to EUR = 1 EUR = $1.14. Therefore, the cost of a cappucino at Pret a Manger in Berlin Mitte is approximately $1.14.


### Source knowledging (manual)

In [10]:
llmchain_information = [
    "A LLMChain is the most common type of chain. It consists of a PromptTemplate, a model (either an LLM or a ChatModel), and an optional output parser. This chain takes multiple input variables, uses the PromptTemplate to format them into a prompt. It then passes that to the model. Finally, it uses the OutputParser (if provided) to parse the output of the LLM into a final format.",
    "Chains is an incredibly generic concept which returns to a sequence of modular components (or other chains) combined in a particular way to accomplish a common use case.",
    "LangChain is a framework for developing applications powered by language models. We believe that the most powerful and differentiated applications will not only call out to a language model via an api, but will also: (1) Be data-aware: connect a language model to other sources of data, (2) Be agentic: Allow a language model to interact with its environment. As such, the LangChain framework is designed with the objective in mind to enable those types of applications."
]

source_knowledge = "\n".join(llmchain_information)

In [11]:
template_with_context="""You are a helpful assistant that answers questions of the user, using the context provided below.

Contexts:{source_knowledge}

{human_message}
"""

prompt2=PromptTemplate(template=template_with_context, input_variables=["human_message",  "source_knowledge"])

In [12]:
print(prompt2.format(human_message="What is a LLMChain?", source_knowledge=source_knowledge))

You are a helpful assistant that answers questions of the user, using the context provided below.

Contexts:A LLMChain is the most common type of chain. It consists of a PromptTemplate, a model (either an LLM or a ChatModel), and an optional output parser. This chain takes multiple input variables, uses the PromptTemplate to format them into a prompt. It then passes that to the model. Finally, it uses the OutputParser (if provided) to parse the output of the LLM into a final format.
Chains is an incredibly generic concept which returns to a sequence of modular components (or other chains) combined in a particular way to accomplish a common use case.
LangChain is a framework for developing applications powered by language models. We believe that the most powerful and differentiated applications will not only call out to a language model via an api, but will also: (1) Be data-aware: connect a language model to other sources of data, (2) Be agentic: Allow a language model to interact with

In [13]:
context_chain = LLMChain(llm=conv_model, prompt=prompt2, verbose=True)

In [14]:
print(context_chain.run({

  'source_knowledge': source_knowledge,

  'human_message': "What is Langchain?"

}))



> Entering new LLMChain chain...
Prompt after formatting:
You are a helpful assistant that answers questions of the user, using the context provided below.

Contexts:A LLMChain is the most common type of chain. It consists of a PromptTemplate, a model (either an LLM or a ChatModel), and an optional output parser. This chain takes multiple input variables, uses the PromptTemplate to format them into a prompt. It then passes that to the model. Finally, it uses the OutputParser (if provided) to parse the output of the LLM into a final format.
Chains is an incredibly generic concept which returns to a sequence of modular components (or other chains) combined in a particular way to accomplish a common use case.
LangChain is a framework for developing applications powered by language models. We believe that the most powerful and differentiated applications will not only call out to a language model via an api, but will also: (1) Be data-aware: connect a language model to other sources of d

## RAG 
### Create database to store your corpus on

In [15]:
# load dependencies
from langchain.document_loaders import DirectoryLoader
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import Chroma
import shutil

In [16]:
# set params
DATA_PATH = "data/html"
CHROMA_PATH = "chroma_db"
EMBED_MODEL = "all-MiniLM-L6-v2" # Chroma defaults to "sentence-transformers/all-MiniLM-L6-v2"
# alternative: "BAAI/bge-small-en-v1.5"

# Load Documents

In [17]:
from bs4 import SoupStrainer
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import BSHTMLLoader

In [18]:
# define Beautiful Soup key word args
bs_kwargs = {
    "features": "html.parser", 
    "parse_only": SoupStrainer("p") # only include relevant text
}

# define Loader key word args
loader_kwargs = {
    "open_encoding": "utf-8",
    "bs_kwargs": bs_kwargs
}

# define Loader
loader = DirectoryLoader(
    path='data/html', 
    glob="*.html", 
    loader_cls=BSHTMLLoader,
    loader_kwargs=loader_kwargs,
    show_progress=True
    )

# load docs
documents = loader.load()
len(documents)

100%|██████████| 3487/3487 [00:04<00:00, 716.02it/s]


3487

In [19]:
# clean up document content
import re 

for doc in documents:
    doc.page_content = doc.page_content.replace("\n", " ").replace("\t", " ")
    doc.page_content = re.sub("\\s+", " ", doc.page_content)

In [20]:
# inspect first doc
documents[0]

Document(page_content='Warranties of data shall be developed and used in accordance with agency regulations.', metadata={'source': 'data/html/46.708.html', 'title': ''})

## Label Metadata

In [21]:
# # add source label
# for doc in documents:
#     doc_source = re.search("\d{1,2}[.]\d+(\-\d)*", doc.metadata["source"]).group() 
#     doc.metadata["source"] = " ".join(["FAR", doc_source])

In [22]:
# define Beautiful Soup key word args
bs_kwargs = {
    "features": "html.parser", 
    "parse_only": SoupStrainer("title") # only include relevant text
}

# define Loader key word args
loader_kwargs = {
    "open_encoding": "utf-8",
    "bs_kwargs": bs_kwargs
}

loader = DirectoryLoader(
    path='data/html', 
    glob="*.html", 
    loader_cls=BSHTMLLoader,
    loader_kwargs=loader_kwargs,
    show_progress=True
    )

document_titles = loader.load()

# convert source metadata into a list
title_list = [doc.metadata["title"] for doc in document_titles]

# add title label
i = 0
for doc in documents:
    doc.metadata["source"] = " ".join(["FAR", title_list[i]])
    i += 1

  0%|          | 0/3487 [00:00<?, ?it/s]

100%|██████████| 3487/3487 [00:03<00:00, 1107.21it/s]


In [23]:
# # add FAR part label
# import re 

# for doc in docs:
#     doc_part = re.search('^(\d{1,2})', doc.metadata['source']).group()
#     doc.metadata["part"] = " ".join(["FAR Part", doc_part])
    
#     print(doc.metadata["part"])

In [24]:
# inspect metadata 
doc_metadata = [doc.metadata  for doc in documents]
doc_metadata[0:5]

[{'source': 'FAR 46.708 Warranties of data.', 'title': ''},
 {'source': 'FAR 9.405 Effect of listing.', 'title': ''},
 {'source': 'FAR 11.106 Purchase descriptions for service contracts.',
  'title': ''},
 {'source': 'FAR 16.204 Fixed-price incentive contracts.', 'title': ''},
 {'source': 'FAR 7.201 [Reserved]', 'title': ''}]

# Embed Documents & Upload to Vector Database

In [25]:
# define text embedding model
embedding_func = SentenceTransformerEmbeddings(model_name=EMBED_MODEL)

# See https://huggingface.co/spaces/mteb/leaderboard

2023-12-13 23:16:18 - Load pretrained SentenceTransformer: all-MiniLM-L6-v2
2023-12-13 23:16:19 - Use pytorch device: cpu


In [26]:
# first, clear out current db
if os.path.exists(CHROMA_PATH):
    shutil.rmtree(CHROMA_PATH)

# initialize Chroma db and save locally
db = Chroma.from_documents(
    documents=documents, embedding=embedding_func, persist_directory=CHROMA_PATH
    )

db.persist()

# print message
print(f"Saved {len(documents)} chunks to {CHROMA_PATH}.")

2023-12-13 23:16:19 - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.


Batches: 100%|██████████| 109/109 [01:46<00:00,  1.03it/s]


Saved 3487 chunks to chroma_db.


# Query Vector Database

In [27]:
# query vector db
query = "What is the purpose of the Federal Acquisition Regulations?"
matching_docs = db.similarity_search_with_relevance_scores(
    query=query, 
    k=4, # number of docs to return
    score_threshold=.5,
    #filter=[{"":""}]
    )

matching_docs

Batches: 100%|██████████| 1/1 [00:00<00:00, 39.68it/s]


[(Document(page_content='The Federal Acquisition Regulations System is established for the codification and publication of uniform policies and procedures for acquisition by all executive agencies. The Federal Acquisition Regulations System consists of the Federal Acquisition Regulation (FAR), which is the primary document, and agency acquisition regulations that implement or supplement the FAR. The FAR System does not include internal agency guidance of the type described in 1.301(a)(2).', metadata={'source': 'FAR 1.101 Purpose.', 'title': ''}),
  0.677825443885269),
 (Document(page_content='This part sets forth basic policies and general information about the Federal Acquisition Regulations System including purpose, authority, applicability, issuance, arrangement, numbering, dissemination, implementation, supplementation, maintenance, administration, and deviation. subparts 1.2,1.3, and 1.4 prescribe administrative procedures for maintaining the FAR System.', metadata={'source': 'FAR

### Query data from your database based on your prompt

In [28]:
### adapted version 
PROMPT_TEMPLATE = """
Answer the question based only on the following context:

{context}

---

Answer the question based on the above context: {question}
"""

# TODO: verbose arg
# TODO: when results not found, return generic message
def RAG(query_text, verbose=True): 
    # Search the DB.
    results = db.similarity_search_with_relevance_scores(query_text, k=5, score_threshold=.5)
    if len(results) == 0 or results[0][1] < 0.5:
        print(f"Unable to find matching results.")
        return

    context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
    if len(context_text) > 1000:
        context_text = context_text[:1000]
        print("Warning: Context exceeded 1000 characters, trimming from the end.")

    prompt_template=PromptTemplate(template=PROMPT_TEMPLATE, input_variables=["context",  "question"])
    #prompt = prompt_template.format(context=context_text, question=query_text)
    #print(prompt)

    chain = LLMChain(llm=conv_model, prompt=prompt_template, verbose=True)
    response_text = chain.run({"context": context_text, "question": query_text})
    
    sources = [doc.metadata.get("source", None) for doc, _score in results]
    formatted_response = f"Response: {response_text}\nSources: {sources}"
    
    print(formatted_response) if verbose == True else None

    return(response_text)

### Parse the augumented prompt into the chatmodel

In [29]:
RAG("What does the Federal Acquisition Regulations define?")

Batches: 100%|██████████| 1/1 [00:00<00:00, 59.71it/s]



> Entering new LLMChain chain...
Prompt after formatting:

Answer the question based only on the following context:

The Federal Acquisition Regulations System is established for the codification and publication of uniform policies and procedures for acquisition by all executive agencies. The Federal Acquisition Regulations System consists of the Federal Acquisition Regulation (FAR), which is the primary document, and agency acquisition regulations that implement or supplement the FAR. The FAR System does not include internal agency guidance of the type described in 1.301(a)(2).

---

This part sets forth basic policies and general information about the Federal Acquisition Regulations System including purpose, authority, applicability, issuance, arrangement, numbering, dissemination, implementation, supplementation, maintenance, administration, and deviation. subparts 1.2,1.3, and 1.4 prescribe administrative procedures for maintaining the FAR System.

---

Agency acquisition regulat


> Finished chain.
Response: The Federal Acquisition Regulations define the Federal Acquisition Regulations System to include internal agency guidance, which is not subject to the FAR. This includes rules, regulations, orders, and policies that an agency enters into, whether or not they are codified within the FAR.
Sources: ['FAR 1.101 Purpose.', 'FAR 1.000 Scope of part.', 'FAR 1.302 Limitations.']


'The Federal Acquisition Regulations define the Federal Acquisition Regulations System to include internal agency guidance, which is not subject to the FAR. This includes rules, regulations, orders, and policies that an agency enters into, whether or not they are codified within the FAR.'

### Human evaluation of RAG model
To get an idea of whether our model is working as intended, we've selected 50 example questions based on training material prepared for government officials training to obtaining their Contracting Officer certification, requiring the ability to recall detailed information from the FAR.

In [30]:
import pandas as pd

# load spreadsheet
df = pd.read_excel('data/xlsx/RAG_evaluation_input.xlsx')

# preview 
df.head(5)

,question,question_edited,answer,answer_edited,response_no_context,response_context,source
0,True or False? Contractors who provide product...,Is it true that contractors who provide produc...,True. Both FAR 1.102(b)(c) and FAR 1.102-3 not...,True. Both FAR 1.102(b)(c) and FAR 1.102-3 not...,NaN,NaN,https://publiccontractinginstitute.com/far-kno...
1,Who are the principal customers for the produc...,Who are the principal customers for the produc...,FAR 1.102-2 provides that the principal custom...,FAR 1.102-2 provides that the principal custom...,NaN,NaN,https://publiccontractinginstitute.com/far-kno...
2,True or False? A contracting officer may unila...,Can a contracting officer unilaterally incorpo...,False. FAR 1.108(d)(3) provides that “Contract...,False. FAR 1.108(d)(3) provides that “Contract...,NaN,NaN,https://publiccontractinginstitute.com/far-kno...
3,What three government officials are personally...,What three government officials are personally...,FAR 1.202 provides that “Agency compliance wit...,FAR 1.202 provides that “Agency compliance wit...,NaN,NaN,https://publiccontractinginstitute.com/far-kno...
4,What is a D&F?,What is a D&F?,FAR 1.701 provides that a “Determination and F...,FAR 1.701 provides that a “Determination and F...,NaN,NaN,https://publiccontractinginstitute.com/far-kno...


In [31]:
# number of questions & answers
df.shape[0]

50

We feed our list of questions into our RAG pipeline and record the responses.

In [32]:
# TODO: Add verbose arg to RAG()

# create list of questions
question_list = df["question_edited"]

# create list of responses
response_list = [RAG(question) for question in question_list]

# create list of baseline responses
baseline_response_list = [conv_chain.run(question) for question in question_list]

Batches: 100%|██████████| 1/1 [00:00<00:00, 25.03it/s]
/Users/steve/.local/share/virtualenvs/dl-tutorial-Si6gq9gB/lib/python3.11/site-packages/langchain_core/vectorstores.py:325: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.5
  warnings.warn(


Unable to find matching results.


Batches: 100%|██████████| 1/1 [00:00<00:00, 77.26it/s]




> Entering new LLMChain chain...
Prompt after formatting:

Answer the question based only on the following context:

 (a) The vision for the Federal Acquisition System is to deliver on a timely basis the best value product or service to the customer, while maintaining the public’s trust and fulfilling public policy objectives. Participants in the acquisition process should work together as a team and should be empowered to make decisions within their area of responsibility. (b) The Federal Acquisition System will- (1) Satisfy the customer in terms of cost, quality, and timeliness of the delivered product or service by, for example- (i) Maximizing the use of commercial products and commercial services; (ii) Using contractors who have a track record of successful past performance or who demonstrate a current superior ability to perform; and (iii) Promoting competition; (2) Minimize administrative operating costs; (3) Conduct business with integrity, fairness, and openness; and (4) Fulf

Batches: 100%|██████████| 1/1 [00:00<00:00, 46.15it/s]

...

Batches: 100%|██████████| 1/1 [00:00<00:00, 70.85it/s]
/Users/steve/.local/share/virtualenvs/dl-tutorial-Si6gq9gB/lib/python3.11/site-packages/langchain_core/vectorstores.py:325: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.5
  warnings.warn(


Unable to find matching results.


Batches: 100%|██████████| 1/1 [00:00<00:00, 69.94it/s]



> Entering new LLMChain chain...
Prompt after formatting:

Answer the question based only on the following context:

The contracting officer shall cooperate with Department of Labor representatives in the examination of records, interviews with service employees, and all other aspects of investigations undertaken by the Department. When asked, agencies shall furnish the Wage and Hour Administrator or a designee, any available information on contractors, subcontractors, their contracts, and the nature of the contract services. The contracting officer shall promptly refer, in writing to the appropriate regional office of the Department, apparent violations and complaints received. Employee complaints shall not be disclosed to the employer.

---

Contracting officers are responsible for ensuring performance of all necessary actions for effective contracting, ensuring compliance with the terms of the contract, and safeguarding the interests of the United States in its contractual relatio


> Finished chain.
Response: The roles and responsibilities of the contracting officer in government contracting include: (1) ensuring compliance with the terms of the contract; (2) safeguarding the interests of the United States in its contractual relationships; (3) performing all necessary actions for effective contracting; and (4) ensuring employee complaints are appropriately referred.
Sources: ['FAR 22.1024 Cooperation with the Department of Labor.', 'FAR 1.602-2 Responsibilities.', 'FAR 5.403 Requests from Members of Congress.', 'FAR 1.601 General.', 'FAR 1.602-1 Authority.']


> Entering new LLMChain chain...
Prompt after formatting:
You are a helpful assistant that answers questions of the user.
Is it true that contractors who provide products or services to the Federal Government are to be considered members of the Government’s “Federal Acquisition Team”?


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
You are a helpful assistant that answers qu

In [33]:
# TODO: text preprocessing

To systematically evaluate the performance of our model relative to the baseline, we will convert our questions and responses (both those provided with and without context) to text embeddings. We will then calculate the cosine similarity between the human-generated responses and the AI-generated responses. We will calculate this metric for both those AI-generated responses which used context as well as those which did not.

In [34]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

2023-12-13 23:20:49 - Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2
2023-12-13 23:20:50 - Use pytorch device: cpu


For consistency, we use the same text embedding model as earlier. 

In [35]:
# create a list of answers
answer_list = df["answer_edited"]

# compute answer embeddings
answer_embeddings = [embedding_model.encode(answer) for answer in answer_list]

# compute embeddings for responses with context
response_embeddings = []
for response in response_list:
    if response is None:
        embedding = pd.NA
    else:
        embedding = embedding_model.encode(response)
    response_embeddings.append(embedding)

# compute embeddings for responses without context
baseline_response_embeddings = [] 
for response in baseline_response_list:
    if response is None:
        embedding = pd.NA
    else:
        embedding = embedding_model.encode(response)
    baseline_response_embeddings.append(embedding)

Batches: 100%|██████████| 1/1 [00:00<00:00, 43.99it/s]


In [36]:
import numpy as np 

# reshape embeddings to 2D arrays (required by cosine_similarity)
answer_embeddings = [np.reshape(embedding, (1, -1)) for embedding in answer_embeddings]
response_embeddings = [np.reshape(embedding, (1, -1)) for embedding in response_embeddings]
baseline_response_embeddings = [np.reshape(embedding, (1, -1)) for embedding in baseline_response_embeddings]

In [87]:
from sklearn.metrics.pairwise import cosine_similarity

# calculate similarity scores
# questions vs responses with context
similarity_scores = []
for a_embedding, r_embedding in zip(answer_embeddings, response_embeddings):
    if pd.isna(r_embedding).any():
        score = pd.NA
    else:
        score = cosine_similarity(a_embedding, r_embedding).item()
    similarity_scores.append(score)

# questions vs responses without context
baseline_similarity_scores = []
for a_embedding, br_embedding in zip(answer_embeddings, baseline_response_embeddings):
    if pd.isna(br_embedding).any():
        score = pd.NA
    else:
        score = cosine_similarity(a_embedding, br_embedding).item()
    baseline_similarity_scores.append(score)

In [90]:
# create summary table
summary_df = pd.DataFrame({
    "model": ["Baseline", "RAG"],
    "avg_similarity_score": [pd.Series(baseline_similarity_scores).mean(skipna=True), 
                             pd.Series(similarity_scores).mean(skipna=True)],
    "response_rate": [(pd.Series(baseline_similarity_scores).count() / len(baseline_similarity_scores) * 100), 
                    (pd.Series(similarity_scores).count() / len(similarity_scores) * 100)]
                    })

summary_df

,model,avg_similarity_score,response_rate
0,Baseline,0.295204,100.0
1,RAG,0.392347,66.0


Lastly, we save our results to an excel spreadsheet so that they can be easily evaluated (no Python required!).

In [33]:
# add responses and similarity scores to df
df["response_context"] = response_list
df["response_no_context"] = baseline_response_list
df["similarity_score_no_context"] = baseline_similarity_scores
df["similarity_score_context"] = similarity_scores

# save xlslx
df.to_excel("data/xlsx/RAG_evaluation_output.xlsx")

# inspect df
df.head(5)

,question,question_edited,answer,response_no_context,response_context,source
0,True or False? Contractors who provide product...,Is it true that contractors who provide produc...,True. Both FAR 1.102(b)(c) and FAR 1.102-3 not...,"As an AI language model, I'm not entirely sure...",None,https://publiccontractinginstitute.com/far-kno...
1,Who are the principal customers for the produc...,Who are the principal customers for the produc...,FAR 1.102-2 provides that the principal custom...,The principal customers for the products and s...,The principal customers for the products and s...,https://publiccontractinginstitute.com/far-kno...
2,True or False? A contracting officer may unila...,Can a contracting officer unilaterally incorpo...,False. FAR 1.108(d)(3) provides that “Contract...,"No, a contracting officer cannot unilaterally ...","\nThe answer to this question is yes, a contra...",https://publiccontractinginstitute.com/far-kno...
3,What three government officials are personally...,What three government officials are personally...,FAR 1.202 provides that “Agency compliance wit...,The three government officials responsible for...,None,https://publiccontractinginstitute.com/far-kno...
4,What is a D&F?,What is a D&F?,FAR 1.701 provides that a “Determination and F...,"Sorry, I cannot process this search term as it...",None,https://publiccontractinginstitute.com/far-kno...


## References

https://github.com/pinecone-io/examples/blob/master/learn/generation/langchain/rag-chatbot.ipynb <br>
https://github.com/pixegami/langchain-rag-tutorial/tree/main <br>
https://www.youtube.com/watch?v=LhnCsygAvzY <br>
https://www.youtube.com/watch?v=tcqEUSNCn8I <br>
https://towardsdatascience.com/a-3-step-approach-to-evaluate-a-retrieval-augmented-generation-rag-5acf2aba86de <br>
https://github.com/curiousily/Get-Things-Done-with-Prompt-Engineering-and-LangChain <br>
https://www.mlexpert.io/prompt-engineering/chatbot-with-local-llm-using-langchain <br>
https://www.youtube.com/watch?v=N7dGOUwufBM <br>
https://www.youtube.com/watch?v=ypzmPwLH_Q4&list=PLIUOU7oqGTLjAwPzyCu6m0wxLOlhJg8N5&index=2<br>
https://www.youtube.com/watch?v=qMIM7dECAkc <br>
https://www.youtube.com/watch?v=ukj_ITJKBwE&list=PLIUOU7oqGTLjAwPzyCu6m0wxLOlhJg8N5&index=6
https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2
